In [2]:
# we import.
# we enable caching.
# we load seasaon and races.
# we create two empty lists for all laps and for the messages for trakc status.


import fastf1
import pandas as pd





seasons = [2022, 2023]
races_per_season = {2022: 21, 2023: 21}

all_laps = []
all_messages = []


In [ ]:
import fastf1
import pandas as pd

fastf1.Cache.enable_cache('cache')

seasons = [2023]
races_per_season = {2023: 22}

all_laps = []
all_messages = []
all_telemetry = []

for year in seasons:
    for round_num in range(1, races_per_season[year] + 1):
        try:
            session = fastf1.get_session(year, round_num, 'R')
            session.load(laps=True, telemetry=True, messages=True, weather=True)
        except Exception as e:
            print(f"Skipped {year} Round {round_num}: {e}")
            continue

        laps = session.laps.copy()
        messages = session.race_control_messages.copy()
        weather = session.weather_data.copy()

        # Merge weather into laps
        if not laps.empty and not weather.empty:
            laps = pd.merge_asof(
                laps.sort_values('LapStartTime'),
                weather.sort_values('Time'),
                left_on='LapStartTime',
                right_on='Time',
                direction='backward'
            )

        # Add year and round info
        laps['Year'] = year
        laps['Round'] = round_num
        messages['Year'] = year
        messages['Round'] = round_num

        # -----------------------------
        # Resample telemetry every 20 seconds
        # -----------------------------
        for idx, lap in session.laps.iterlaps():
            try:
                lap_telem = lap.get_telemetry()
                if lap_telem.empty:
                    continue

                # Create a lap-relative time column in seconds
                lap_telem['LapTimeSec'] = lap_telem['Time'] - lap['LapStartTime']
                lap_telem.set_index('LapTimeSec', inplace=True)

                # Resample in 20-second windows
                lap_telem_resampled = lap_telem.resample('20S').agg({
                    'Speed': 'mean',
                    'Throttle': 'mean',
                    'Brake': 'max',
                    'nGear': 'max',
                    'Distance': 'max',
                    'RelativeDistance': 'max'
                })

                # Add identifying info
                lap_telem_resampled['LapNumber'] = lap.LapNumber
                lap_telem_resampled['Driver'] = lap['Driver']
                lap_telem_resampled['Year'] = year
                lap_telem_resampled['Round'] = round_num

                all_telemetry.append(lap_telem_resampled)

            except Exception as e:
                print(f"Telemetry failed for lap {idx}: {e}")

        all_laps.append(laps)
        all_messages.append(messages)

# Combine all data
all_laps_df = pd.concat(all_laps, ignore_index=True)
all_messages_df = pd.concat(all_messages, ignore_index=True)
all_telemetry_df = pd.concat(all_telemetry, ignore_index=False)  # Keep index as time

# Save to CSV
all_laps_df.to_csv('all_races_laps.csv', index=False)
all_messages_df.to_csv('race_control_messages.csv', index=False)
all_telemetry_df.to_csv('telemetry_20s.csv')

print(f"Saved all laps ({len(all_laps_df)})")
print(f"Saved all messages ({len(all_messages_df)})")
print(f"Saved all telemetry segments ({len(all_telemetry_df)})")


In [ ]:
# drop unnecesary columns 
cols_to_drop = [ 'SpeedFL', 
                'IsPersonalBest','PitOutTime', 'PitInTime']
all_laps_df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
all_laps_df['AvgSectorSpeed'] = all_laps_df[['SpeedI1', 'SpeedI2', 'SpeedST']].mean(axis=1)
# Save to CSV
all_laps_df.to_csv('all_races_laps.csv', index=False)
all_messages_df.to_csv('race_control_messages.csv', index=False)
print(all_laps_df.columns)
print(f"Saved all laps to 'all_races_laps.csv' ({len(all_laps_df)} rows).")
print(f"Saved all race control messages to 'race_control_messages.csv' ({len(all_messages_df)} rows).")


In [ ]:
import fastf1
import pandas as pd

fastf1.Cache.enable_cache('cache')

session = fastf1.get_session(2023, 1, 'R')
session.load(laps=True, telemetry=True)

lap = session.laps.iloc[0]
lap_telem = lap.get_telemetry()

# Make sure SessionTime is datetime
lap_telem['SessionTime'] = pd.to_datetime(lap_telem['SessionTime'])

# Set index
lap_telem = lap_telem.set_index('SessionTime')

# Resample in 20-second windows
lap_telem_resampled = lap_telem.resample('20S').agg({
    'Speed': 'mean',
    'Throttle': 'mean',
    'Brake': 'max',
    'nGear': 'max',
    'Distance': 'max',
    'RelativeDistance': 'max'
})

print(lap_telem_resampled)


In [4]:
import fastf1
import pandas as pd



# Load a session
session = fastf1.get_session(2023, 1, 'R')
session.load(laps=True, telemetry=True)

# Pick one lap
lap = session.laps.iloc[0]

# Get telemetry
lap_telem = lap.get_telemetry()

# Set the index to SessionTime
lap_telem = lap_telem.set_index('SessionTime')
car_data = lap.get_car_data()
print(car_data.columns)



lap_telem_resampled = lap_telem.resample('20S').agg({
    'Speed': 'mean',
    'Throttle': 'mean',
    'Brake': 'max',
    'nGear': 'mean',
    'Distance': 'max',
    'RelativeDistance': 'max'
})


print(lap_telem_resampled.columns)


req         WARNING 	DEFAULT CACHE ENABLED! (352.88 MB) /home/abdul/.cache/fastf1
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data.

Index(['Date', 'RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Source',
       'Time', 'SessionTime'],
      dtype='object')
Index(['Speed', 'Throttle', 'Brake', 'nGear', 'Distance', 'RelativeDistance'], dtype='object')


/tmp/ipykernel_271590/2434034981.py:23: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  lap_telem_resampled = lap_telem.resample('20S').agg({
